In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import re
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


In [37]:
def clean_text(text):
    text = text.replace('[','').replace(']', '')
    text = re.sub("(<li>)|(</li>)|</p>|<p>|\xa0|<ul>|</ul>|'|-| +|,| ,", ' ', text)
    text = re.sub(" +", " ", text)
    return text

In [38]:
def extract_ku(fag):
    
    headers = {'User-Agent':'bwt973@alumni.ku.dk'}
    
    url = f"https://studier.ku.dk/kandidat/{fag}/faglig-profil-og-job/"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    return soup

def transform_ku(soup):

    divs = soup.find_all("div", class_="col-xs-12 col-sm-8 col-md-6 main-content")

    text_ku =[]

    for i in range(len(divs)):
        text_ku.append(divs[i].find_all("p"))

    divs2 = soup.find_all("div", class_="col-xs-12 col-sm-8 col-md-6 main-content")

    for i in range(len(divs2)):
        text_ku.append(divs2[i].find_all("ul"))
        text_ku = clean_text(text_ku)
    
    return text_ku

ku_list = []

search_list = ['psykologi', 'sociologi', 'statskundskab', 'antropologi', 'oekonomi']


for k in search_list:
    soup = extract_ku(k)
    ku_list.append(transform_ku(soup))

ku_df = pd.DataFrame(data=ku_list)

#Merging the two colums 
ku_df=ku_df[0]+ku_df[1]

#Making the object a dataframe
ku_df=pd.DataFrame(ku_df)

ku_df=pd.DataFrame.transpose(ku_df)

ku_df.columns=['cand.psych_ku', 'cand.scient.soc_ku', 'cand.scient.pol_ku', 'cand.scient.anth_ku', 'cand.oecon_ku']

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
def extract_au(fag):
    
    headers = {'User-Agent':'tps798@alumni.ku.dk'}
    
    url = f"https://bachelor.au.dk/{fag}"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    return soup

In [ ]:
soup = extract_au('statskundskab')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_stats = text[75:77]

stats_df = pd.DataFrame(data=text_stats, columns=['cand.scient.pol_au'])
stats = pd.DataFrame([', '.join(stats_df['cand.scient.pol_au'].to_list())], columns=['cand.scient.pol_au'])

In [ ]:
soup = extract_au('oekonomi')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_oek = text[60:62]

oek_df = pd.DataFrame(data=text_oek, columns=['cand.oecon_au'])
oek = pd.DataFrame([', '.join(oek_df['cand.oecon_au'].to_list())], columns=['cand.oecon_au'])

In [ ]:
soup = extract_au('antropologi')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_ant = text[50:55]

ant_df = pd.DataFrame(data=text_ant, columns=['cand.scient.anth_au'])
ant = pd.DataFrame([', '.join(ant_df['cand.scient.anth_au'].to_list())], columns=['cand.scient.anth_au'])

In [39]:
soup = extract_au('psykologi')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_psyk = text[74:78]

psyk_df = pd.DataFrame(data=text_psyk, columns=['cand.psych_au'])
psyk = pd.DataFrame([', '.join(psyk_df['cand.psych_au'].to_list())], columns=['cand.psych_au'])

In [40]:
frames = [ant, stats, psyk, oek]

au_df = pd.concat(frames, axis=1)

In [41]:
def extract_aau(fag):
    
    headers = {'User-Agent':'bwt973@alumni.ku.dk'}
    
    url = f"https://www.aau.dk/uddannelser/kandidat/{fag}"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    return soup

def transform_aau(soup):

    divs = soup.find_all("main", class_="Main_Main__2KIvG")

    for item in divs:
        text_aau = item.find_all()[0].text.strip()

        aau_text = {
            "text_aau" : text_aau, 
        }
        aau_list.append(aau_text)

        text_aau = clean_text(text_aau)
        
    return text_aau

aau_list = []

search_list = ['psykologi', 'sociologi', 'oekonomi']

for k in search_list:
    try: 
        soup = extract_aau(k)
        transform_aau(soup)
    except:
        break

aau_df = pd.DataFrame(data=aau_list)

aau_df=pd.DataFrame.transpose(aau_df)

aau_df.columns=['cand.psych_aau', 'cand.scient.soc_aau', 'cand.oecon_aau']

aau_df = aau_df.reset_index(drop=True)

In [42]:
merge_frames = [ku_df, au_df, aau_df]

combined_df = pd.concat(merge_frames, axis=1)

combined_df['cand.psych_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.psych_ku']]
combined_df['cand.scient.pol_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.scient.pol_ku']]
combined_df['cand.oecon_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.oecon_ku']]
combined_df['cand.scient.anth_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.scient.anth_ku']]
combined_df['cand.scient.soc_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.scient.soc_ku']]

combined_df['cand.psych'] = combined_df['cand.psych_aau'] + combined_df['cand.psych_au'] + combined_df['cand.psych_ku_string']
combined_df['cand.scient.anth'] = combined_df['cand.scient.anth_au'] + combined_df['cand.scient.anth_ku_string']
combined_df['cand.scient.pol'] = combined_df['cand.scient.pol_au'] + combined_df['cand.scient.pol_ku_string']
combined_df['cand.scient.soc'] = combined_df['cand.scient.soc_aau'] + combined_df['cand.scient.soc_ku_string']
combined_df['cand.oecon'] = combined_df['cand.oecon_aau'] + combined_df['cand.oecon_au'] + combined_df['cand.oecon_ku_string']

In [43]:
combined_df.drop(['cand.psych_aau', 'cand.scient.soc_aau', 'cand.oecon_aau', 'cand.psych_au', \
                  'cand.scient.anth_au', 'cand.scient.pol_au','cand.oecon_au', \
                  'cand.psych_ku', 'cand.scient.soc_ku', 'cand.scient.pol_ku', 'cand.scient.anth_ku', \
                  'cand.oecon_ku', 'cand.psych_ku_string', 'cand.scient.pol_ku_string', 'cand.oecon_ku_string', \
                  'cand.scient.anth_ku_string', 'cand.scient.soc_ku_string'], axis=1, inplace=True)

In [44]:
university_df = combined_df
university_df['cand.scient.soc'].loc[0]

'KandidatSociologiUddannelsen bygger videre på den grundlæggende videnskabelige viden om sociologisk teori og metode, som du har opnået på bacheloruddannelsen.Få kendskab tilArbejdsliv, organisation og Human Resource Management, Sociale fællesskaber og sociale forskelleSundhed, sygdom, steder og rum Hverdagsliv, kultur og samtidsdiagnoserAalborg2-årig uddannelseDanskSøg optagelseKandidatSociologiUddannelsen bygger videre på den grundlæggende videnskabelige viden om sociologisk teori og metode, som du har opnået på bacheloruddannelsen.Få kendskab tilArbejdsliv, organisation og Human Resource Management, Sociale fællesskaber og sociale forskelleSundhed, sygdom, steder og rum Hverdagsliv, kultur og samtidsdiagnoserAalborg2-årig uddannelseDanskSøg optagelseKandidatuddannelsen sigter mod at skærpe din personlige, faglige profil. På kandidatuddannelsen vælger du mellem valgfrie moduler i videregående teori og metode, sociologiske specialiseringer og valgfag eller praktik. Et praktikophold ka